In [1]:
import pandas as pd
import re
import string
from vncorenlp import VnCoreNLP
!pip install networkx
!pip install googletrans
from googletrans import Translator
import networkx as nx
from itertools import combinations
import matplotlib.pyplot as plt
import nltk
from nltk import FreqDist
from collections import Counter

In [24]:
import pandas as pd
# !pip install vncorenlp
data = pd.read_excel('data_new.xlsx',index_col=None,na_values=['NA'],usecols="B")
sample_data = data[:]

In [25]:
len(sample_data)

41675

In [3]:
import io
stop_words = [line.strip() for line in open('vietnamese-stopwords.txt','r', encoding ="utf8")]
stop_words = set([w.lower().replace(' ','_') for w in stop_words])

In [4]:
annotator = VnCoreNLP("vncore_nlp/VnCoreNLP-1.1.1.jar", annotators="wseg,pos", max_heap_size='-Xmx4g')

In [5]:
WEB_ADDRESS_RE = r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}     /)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))'

In [6]:
def get_clean_sentences(content):
    content = re.sub(WEB_ADDRESS_RE, ' ', content)
    content = content.strip().split('\n')
    content = [string for string in content if string != ""]
    clean_sentences = []
    for p in content:
        sentences = annotator.annotate(p)['sentences']
        for s in sentences:
            words = [word['form'].lower() for word in s if word['posTag'].startswith('N')]
            words = [w for w in words if w not in string.punctuation and w not in stop_words]
            clean_sentences.append(words)
    return clean_sentences

In [28]:
# clean data
start=datetime.now()
clean_sentences=[]
for _, row in sample_data.iterrows():
    clean_sentence = get_clean_sentences(row['content'])
    clean_sentences.append(clean_sentence)
clean_sentences
print(datetime.now()-start)

2:49:44.075093


In [82]:
# make list
start=datetime.now()
words =[]
for sentences in clean_sentences:
    for sentence in sentences:
        words.append(sentence)
print(datetime.now()-start)

0:00:14.059293


In [83]:
#define most common word
from datetime import datetime
start=datetime.now()
common=[]
for sentence in words:
    for word in sentence:
        common.append(word)
most_common = Counter(common).most_common(50)
print(datetime.now()-start)
# set(list(dict(most_common).keys()))
print(most_common)

0:00:15.982242
[('ca', 346804), ('covid-19', 221029), ('bệnh_nhân', 179650), ('y_tế', 110103), ('đà_nẵng', 104364), ('bệnh', 101863), ('dịch', 94724), ('bệnh_viện', 87116), ('việt_nam', 65238), ('trường_hợp', 63474), ('bn', 53331), ('tỉnh', 47321), ('sars-cov', 40158), ('hà_nội', 40128), ('quốc_gia', 38155), ('kết_quả', 34640), ('mỹ', 34186), ('phòng', 33839), ('hiện', 33738), ('cộng_đồng', 31807), ('thế_giới', 31301), ('thành_phố', 31278), ('dịch_bệnh', 30812), ('cơ_sở', 30722), ('tổng_số', 29718), ('mẫu', 29677), ('ban', 29232), ('virus', 27078), ('quảng_nam', 26292), ('nam', 25232), ('người_dân', 24062), ('ảnh', 22741), ('tp', 22186), ('trung_tâm', 21785), ('sức_khoẻ', 21369), ('khu_vực', 21206), ('biện_pháp', 21149), ('hải_dương', 20946), ('huyện', 20248), ('thông_tin', 18987), ('tình_hình', 18688), ('chiều', 18637), ('âm_tính', 18152), ('trung_ương', 17194), ('địa_phương', 16762), ('khẩu_trang', 16568), ('xã_hội', 15828), ('tp.', 15590), ('nhân_viên', 15463), ('chỉ_đạo', 15427)]


In [95]:
for j in most_common:
    print(j)

('ca', 346804)
('covid-19', 221029)
('bệnh_nhân', 179650)
('y_tế', 110103)
('đà_nẵng', 104364)
('bệnh', 101863)
('dịch', 94724)
('bệnh_viện', 87116)
('việt_nam', 65238)
('trường_hợp', 63474)
('bn', 53331)
('tỉnh', 47321)
('sars-cov', 40158)
('hà_nội', 40128)
('quốc_gia', 38155)
('kết_quả', 34640)
('mỹ', 34186)
('phòng', 33839)
('hiện', 33738)
('cộng_đồng', 31807)
('thế_giới', 31301)
('thành_phố', 31278)
('dịch_bệnh', 30812)
('cơ_sở', 30722)
('tổng_số', 29718)
('mẫu', 29677)
('ban', 29232)
('virus', 27078)
('quảng_nam', 26292)
('nam', 25232)
('người_dân', 24062)
('ảnh', 22741)
('tp', 22186)
('trung_tâm', 21785)
('sức_khoẻ', 21369)
('khu_vực', 21206)
('biện_pháp', 21149)
('hải_dương', 20946)
('huyện', 20248)
('thông_tin', 18987)
('tình_hình', 18688)
('chiều', 18637)
('âm_tính', 18152)
('trung_ương', 17194)
('địa_phương', 16762)
('khẩu_trang', 16568)
('xã_hội', 15828)
('tp.', 15590)
('nhân_viên', 15463)
('chỉ_đạo', 15427)


In [90]:
#count #co-occurences
co_occurence_graph = {}
start=datetime.now()
for word in words:
    token_pairs = combinations(word,2)
    for u, v in token_pairs:
        if u in set(list(dict(most_common).keys())) and v in set(list(dict(most_common).keys())):
            if u != v:
                if u in co_occurence_graph:
                    if v in co_occurence_graph[u]:
                        co_occurence_graph[u][v]['weight'] += 1
                    else:
                        co_occurence_graph[u][v] = {'weight':1}
                else:
                    co_occurence_graph[u] = {v:{'weight':1}}
print(datetime.now()-start)

0:03:21.207972


In [91]:
# convert to networkx graph                    
co_occurence_graph = nx.from_dict_of_dicts(co_occurence_graph)

In [92]:
# run k-core procedure
g = nx.k_core(co_occurence_graph,k=5)

In [43]:
#export to gephi input
nx.write_gexf(co_occurence_graph, "period_all.gexf")
print("Trans doo quas")

Trans doo quas


In [93]:
# export nodes
nodes=[]
for node in co_occurence_graph.nodes:
    nodes.append(node)
nodes_all = open("nodes.txt", "w",encoding="utf8")
for element in nodes:
    nodes_all.write(element+'\n')
nodes_all.close()

In [94]:
# export edges
nx.write_edgelist(co_occurence_graph, "edges_all.txt", delimiter=' ', data=['weight'])